In [1]:
import random
import re
import datasets
from nltk import trigrams
from nltk import word_tokenize
from collections import defaultdict
import nltk
nltk.download('punkt')
nltk.download('pertluniprops')
nltk.download('nonbreaking_prefixes')

C:\Users\bebog\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bebog\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading pertluniprops: Package 'pertluniprops' not
[nltk_data]     found in index
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     C:\Users\bebog\AppData\Roaming\nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


True

In [2]:
def keep_arabic_text(text):
    
    arabic_pattern = re.compile('[\u0600-\u06FF\s]+')
    arabic_text = re.findall(arabic_pattern, text)
    arabic_text = ' '.join(arabic_text)
    arabic_text = re.sub(r'[\xa0]', '', arabic_text)
    
    return arabic_text

In [3]:
data = datasets.load_dataset('nthngdy/oscar-mini', name='unshuffled_deduplicated_ar')
len(data['train'])

122375

In [4]:
print(data['train']['text'][0])
print(type(data['train']['text']))

مرحبا بك عزيز الزائر نتمنى لك أوقاتاً سعيدة معنا وأن نزداد شرفا بخدمتك ولا تنسى التسجيل معنا لتستفيد بكل جديد
<class 'list'>


In [5]:
all_text = list(map(keep_arabic_text, data['train']['text']))
all_text[0]

'مرحبا بك عزيز الزائر نتمنى لك أوقاتاً سعيدة معنا وأن نزداد شرفا بخدمتك ولا تنسى التسجيل معنا لتستفيد بكل جديد'

In [6]:
def build_trigram_lm(corpus):
    lm = defaultdict(lambda: defaultdict(lambda: 0))
    for sent in corpus:
        sent = word_tokenize(sent)
        for w1, w2, w3 in trigrams(sent):
            lm[(w1, w2)][w3] += 1
    
    for w1_w2 in lm:
        total_sum = float(sum(lm[w1_w2].values()))
        for w3 in lm[w1_w2]:
            lm[w1_w2][w3] = lm[w1_w2][w3] / total_sum
    
    return lm

In [7]:
def sample_lm(lm):
    text = list(random.choice(list(lm.keys())))
    sent_len = random.randint(10, 50)
    prob = 1.0
    sent_finished = False
    while not sent_finished:
        r = random.random()
        accumulator = .0

        for word in lm[tuple(text[-2:])].keys():
            accumulator += lm[tuple(text[-2:])][word]

            if accumulator >= r:
                prob *= lm[tuple(text[-2:])][word]
                text.append(word)
                break
        if len(text)>=sent_len or text[-2:]==[None, None]:
            sent_finished = True
    return text, prob

In [8]:
lm = build_trigram_lm(all_text)

In [9]:
print(lm[('مرحبا', 'بك')]['في'])
text = sample_lm(lm)
print(' '.join([t for t in text[0] if t]))

0.1
المحتلة الى العدو الإسرائيلي الذي أدّى إلى حدوث تَطّور سريع، حسب تأكيد النائبة البرلمانية سروة عبد الواحد الدقاق ، والفقيه الدستوري يحيى الجمل، وأستاذ القانون الدستوري، يرى في عينها اليمنى بعد الولادة، عن ابى دريد
